In [87]:
import cv2
import numpy as np 
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import matplotlib.image as image
from PIL import Image, ImageOps
from numpy import asarray
import threading

In [88]:
mp_holistic = mp.solutions.holistic #Holistic model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities

In [89]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#Color conversion BGR 2 RGB
    image.flags.writeable = False #Image is unwriteable
    results = model.process(image)#Make prediction
    image.flags.writeable = True#Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#color conversion RGB 2 BGR
    return image, results

In [90]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)#Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)#Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)#Draw hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)#Draw hand connections

In [91]:
def draw_styled_landmarks(image, results):
    #Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness = 1, circle_radius =1),
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness = 1, circle_radius =1)
                             )
    #Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness = 2, circle_radius =2)
                             )
    #Draw hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness = 2, circle_radius =2)
                             )
    #Draw hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness = 2, circle_radius =2)
                             )

In [94]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if \
        results.face_landmarks else np.zeros(468*3)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if \
        results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if \
        results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if \
        results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [95]:
results_test = extract_keypoints(results)

In [96]:
results_test.shape

(1662,)

# setup folder for collection

In [97]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_60FPS')

# Actions that we try to detect# n_inputs = 30, input = 30 frames
actions = np.array(['Walk_', 'Wait_', 'Hello_L','Hello_R','Name_','No_','Sorry_R','Thanks_L',
                    'Fire_','Give up_','House_','Ready_','Drink_','Sleep_','Where_'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

# 6. Preprocess Data and Create Labels and Features

In [98]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [99]:
DATA_PATH

'MP_Data_60FPS'

In [100]:
label_map = {label:num for num, label in enumerate(actions)}

In [101]:
label_map

{'Walk_': 0,
 'Wait_': 1,
 'Hello_L': 2,
 'Hello_R': 3,
 'Name_': 4,
 'No_': 5,
 'Sorry_R': 6,
 'Thanks_L': 7,
 'Fire_': 8,
 'Give up_': 9,
 'House_': 10,
 'Ready_': 11,
 'Drink_': 12,
 'Sleep_': 13,
 'Where_': 14}

In [102]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [103]:
X = np.array(sequences)
X.shape

(900, 30, 1662)

In [104]:
np.array(labels).shape
X.shape

(900, 30, 1662)

In [105]:
y = to_categorical(labels).astype(int)

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
y_test.shape

(90, 15)

In [124]:
X_train.shape

(810, 30, 1662)

In [107]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# 7. Build and Train LSTM Neural Network

In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard
from keras.optimizers import SGD

In [129]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [110]:
del model

In [130]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(actions.shape[0], activation='softmax'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [132]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [133]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
26/26 [==============================] - 8s 24ms/step - loss: 2.7078 - categorical_accuracy: 0.0802
Epoch 2/2000
26/26 [==============================] - 0s 17ms/step - loss: 2.6085 - categorical_accuracy: 0.1346
Epoch 3/2000
26/26 [==============================] - 0s 18ms/step - loss: 2.3407 - categorical_accuracy: 0.1802
Epoch 4/2000
26/26 [==============================] - 0s 18ms/step - loss: 1.9964 - categorical_accuracy: 0.2593
Epoch 5/2000
26/26 [==============================] - 0s 17ms/step - loss: 1.6642 - categorical_accuracy: 0.3506
Epoch 6/2000
26/26 [==============================] - 0s 18ms/step - loss: 1.4032 - categorical_accuracy: 0.4506
Epoch 7/2000
26/26 [==============================] - 0s 17ms/step - loss: 1.1056 - categorical_accuracy: 0.5815
Epoch 8/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.8345 - categorical_accuracy: 0.6519
Epoch 9/2000
26/26 [==============================] - 1s 19ms/step - loss: 0.8619 - categorical_

KeyboardInterrupt: 

In [118]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 dropout_3 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 dropout_4 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [21]:
#tensorboard --logdir=.

In [22]:
res = model.predict(X_test)

3/3 [==============================] - 1s 11ms/step


In [23]:
actions[np.argmax(res[1])]

'ready_'

In [24]:
actions[np.argmax(y_test[1])]

'ready_'

# 9 Save Weights

In [125]:
model.save('finalModel60FPSfinal14.h5')

In [126]:
del model

In [ ]:
model.load_weights('finalModel60fpsfinal11.h5')

# 11 Test in real time

In [116]:
colors = [(0, 215, 255), (192,192,192), (50,127,205), (142, 241, 53), (123, 123 ,133), (214, 122, 141),(214, 122, 141),(214, 122, 141),
          (214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),
         (245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141),(245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141),
         (245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141),(245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    count = 0
    res_arr = np.array(res)
    idx = np.argsort(res_arr) 
    idx = idx[::-1]
    res_arr = res_arr[idx]
    new_actions = actions[idx]
#     for num, prob in enumerate(res):
#         cv2.rectangle(output_frame, (0,60+num*20), (int(prob*100), 90+num*20), colors[num], -1)
#         cv2.putText(output_frame, actions[num].split('_')[0], (0, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
#         cv2.putText(output_frame, str(int(prob*100)), (150, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.putText(output_frame, 'Action', (0, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(output_frame, 'Percentage', (100, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        
    for num, prob in enumerate(sorted(res, reverse= True)):
        count += 1
        cv2.rectangle(output_frame, (0,70+num*20), (int(prob*100), 90+num*20), colors[num], -1)
        cv2.putText(output_frame, new_actions[num].split('_')[0], (0, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(output_frame, str("{:.2f}".format(prob*100))+' %', (110, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        if count == 3:
            break
       
    return output_frame


In [117]:
# 1. New detection variables
sequence = []
sentence = ['']
threshold = 0.75
predictions = []
no_of_frame = 0
a = 0
cap = cv2.VideoCapture(0)
# Set mediapipe model 
start_time = time.time()
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # Read feed
        ret, frame = cap.read()
        no_of_frame += 1
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
        
        #3. Viz logic
            if np.unique(predictions[-15:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    #if len(sentence) > 0: 
                        if actions[np.argmax(res)].split('_')[0] != sentence[-1]: 
                            if actions[np.argmax(res)].split('_')[0] == 'Wait' and sentence[-1]!='':#actions[0].split('_')[0]
                                sentence.append('') #actions[np.argmax(res)].split('_')[0]
                            #elif sentence[-1] == ' ':#actions[0].split('_')[0]
                            elif actions[np.argmax(res)].split('_')[0] != 'Wait' and sentence[-1]=='':
                                sentence.append(actions[np.argmax(res)].split('_')[0])
#                             elif np.unique(predictions[-90:])[0] == np.argmax(res) and a > 4 and actions[np.argmax(res)].split('_')[0] != 'wait':
#                                 sentence.append(actions[np.argmax(res)].split('_')[0])
                            
                   # else:
                       # sentence.append(actions[np.argmax(res)].split('_')[0])

            if len(sentence) > 7: 
                sentence = ['']

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
          
        #cv2.putText(image, "{}".format(no_of_frame), (600, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.rectangle(image, (0,0), (640, 40), (129, 196, 26), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        if no_of_frame == 30:
            no_of_frame = 0
            a += 1
        # Show to screen
        image = cv2.resize(image,(960,720))
        cv2.imshow('OpenCV Feed', image)
        
        # Break gracefully
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
end_time = time.time()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>

1/1 [==============================] - 0s 35ms/step
Thanks_L
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Thanks_L
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
Wait_
<class 'mediapip

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==================

1/1 [==============================] - 0s 29ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Thanks_L
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
Thanks_L
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Thanks_L
<class 'media

...

1/1 [==============================] - 0s 34ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Ready_
<class 'media

1/1 [==============================] - 0s 35ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 46ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
Ready_
<class 'media

1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 45ms/step
Ready_
<class 'media

1/1 [==============================] - 0s 44ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 72ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Sorry_R
<cla

1/1 [==============================] - 0s 32ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
Wait_
<class 'mediapipe.pyth

# 10. Evaluation using Confusion Matrix and Accuracy

In [119]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [120]:
yhat = model.predict(X_test)

3/3 [==============================] - 0s 19ms/step


In [121]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [122]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[83,  0],
        [ 0,  7]],

       [[85,  0],
        [ 0,  5]],

       [[78,  0],
        [ 0, 12]],

       [[88,  0],
        [ 0,  2]],

       [[78,  0],
        [ 0, 12]],

       [[84,  0],
        [ 0,  6]],

       [[88,  0],
        [ 0,  2]],

       [[79,  0],
        [ 0, 11]],

       [[86,  0],
        [ 0,  4]],

       [[81,  0],
        [ 0,  9]],

       [[88,  0],
        [ 0,  2]],

       [[85,  0],
        [ 0,  5]],

       [[87,  0],
        [ 0,  3]],

       [[85,  0],
        [ 0,  5]],

       [[85,  0],
        [ 0,  5]]], dtype=int64)

In [123]:
accuracy_score(ytrue, yhat)

1.0